In [3]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import random
from webdriver_manager.chrome import ChromeDriverManager

## Objetivo

- Descargar de la pagina web del supermercado todos los articulos de la seccion de almacen, tantos sus precios como las ofertas por articulos

## Obtener categorías

En la sección de almacén, los artículos están ordenados por categorías. Por lo tanto, para construir los endpoints, primero se obtiene una lista de todas las categorías de la sección de almacén. La lista de categorías se puede obtener con el siguiente código, que utiliza Selenium:

In [4]:

# Configura Selenium con undetected_chromedriver
options = uc.ChromeOptions()
browser = uc.Chrome(options=options, driver_executable_path=ChromeDriverManager().install())


# Navega a la página web
url = 'https://www.masonline.com.ar/268?map=productClusterIds'
browser.get(url)

# Espera a que se cargue el primer elemento para asegurarte de que la página está cargada
wait = WebDriverWait(browser, 20)
time.sleep(10)
## desplegamos la lista de categoris
browser.find_elements("xpath",'//*[@class="vtex-search-result-3-x-seeMoreButton mt2 pv2 bn pointer c-link"]')[1].click()
wait = WebDriverWait(browser, 5)


# Obtén el contenido HTML completo
html = browser.page_source
soup = BeautifulSoup(html, 'lxml')

categorias = soup.find_all('div',{'class':'vtex-search-result-3-x-filterContent'})[1]
categorias_lista = [categoria.text.strip() for categoria in categorias]
# Imprime el contenido del div
print(categorias_lista)
categorias_lista = categorias_lista[1:-1]
# # Cierra el navegador
browser.quit()


['', 'Golosinas', 'Chocolates', 'Galletitas Dulces', 'Te', 'Yerbas', 'Alfajores', 'Cereales', 'Aderezos', 'Café', 'Mermeladas y untables', 'Especias', 'Sal Y Pimienta', 'Conservas', 'Pastas Secas', 'Endulzantes', 'Aceitunas Y Encurtidos', 'Tostadas, Grisines Y Marineras', 'Galletas De Arroz', 'Aceites', 'Chocolates, Rellenos Y Baños', 'Galletitas Saladas', 'Tomates Y Salsas', 'Harina De Trigo', 'Vinagres Y Aceto', 'Legumbres', 'Sabores En Polvo Y Bolsas Para Horno', 'Bizcochuelos, Brownies Y Tortas', 'Pan Rallado Y Rebozadores', 'Arroz', 'Tortas', 'Postres Y Flanes', 'Sopas', 'Salado', 'Leches Y Chocolatadas', 'Bizcochos', 'Premezclas', 'Pan Lactal', 'Donuts Y Facturas', 'Budines, Magdalenas Y Otros', 'Mas Reposteria', 'Snacks Salados', 'Gelatinas', 'Cacao', 'Pan Para Hamburguesas Y Panchos', 'Harina De Maiz', 'Caldos', 'Pan Árabe, Tortillas Y Otros', 'Pure', 'Panes Del Mundo', 'Budines, Magdalenas Y Piononos', 'Prepizzas Y Pizzetas', 'Otras Harinas', 'Obleas Y Bocaditos', 'Avena', 'Se

Obervación
- Obtuvimos las categorías de la sección de almacén.

## Scrapping por categorías

Este código configura una serie de funciones para realizar el scraping de datos de artículos en una página web utilizando Selenium. Dado que los artículos no se cargan todos de una vez, sino que se muestran de manera dinámica al hacer clic en un botón "Mostrar más" al final de la página, el código automatiza este proceso.

Funcionamiento del Código
Manejo del Botón "Mostrar Más": La función esperar_y_click_en_boton busca y hace clic en el botón "Mostrar más" para cargar más artículos. Este proceso se repite automáticamente hasta que el botón ya no está disponible.

Desplazamiento Automático: La función scroll_to_bottom desplaza la página hacia abajo en pasos para garantizar que todo el contenido adicional sea cargado. Esto se hace mientras se espera que la página se actualice y muestre todos los artículos disponibles.

Extracción de Datos: Una vez que todos los artículos están cargados, el código utiliza BeautifulSoup para analizar el contenido HTML y extraer los datos relevantes, como el nombre, el precio, las imágenes y las ofertas de los artículos.

En resumen, el código maneja la interacción dinámica con la página, incluyendo el desplazamiento automático y la gestión del botón "Mostrar más", para asegurar que todos los artículos sean cargados y extraídos correctamente.

In [6]:
def scroll_to_bottom(browser, pause_time=2, step=100):
    """
    Desplaza la página hacia abajo en pasos hasta que no haya más contenido para cargar.
    Primero vuelve al principio de la página.
    
    :param browser: Instancia del navegador Selenium.
    :param pause_time: Tiempo de espera entre desplazamientos en segundos.
    :param step: Cantidad de píxeles a desplazar en cada paso.
    """
    browser.execute_script("window.scrollTo(0, 0);")  # Vuelve al principio de la página
    time.sleep(pause_time)
    
    last_height = browser.execute_script("return document.body.scrollHeight")
    current_height = 0
    
    while current_height < last_height:
        # Desplázate hacia abajo en pasos
        browser.execute_script(f"window.scrollBy(0, {step});")
        current_height += step
        
        # Espera a que se carguen nuevos elementos
        time.sleep(pause_time)
        
        # Calcula la nueva altura de la página
        new_height = browser.execute_script("return document.body.scrollHeight")
        
        if new_height > last_height:
            last_height = new_height
            

    

def esperar_y_click_en_boton(browser, xpath_del_boton, tiempo_espera=20):
    """
    Espera a que el botón 'Mostrar más' sea clickeable y lo hace clic.
    
    :param browser: Instancia del navegador Selenium.
    :param xpath_del_boton: XPath del botón 'Mostrar más'.
    :param tiempo_espera: Tiempo máximo de espera en segundos.
    """
    try:
        boton_mostrar_mas = WebDriverWait(browser, tiempo_espera).until(
            EC.element_to_be_clickable((By.XPATH, xpath_del_boton))
        )
        boton_mostrar_mas.click()
        print("Hizo clic en el botón 'Mostrar más'.")
        return True
    except (TimeoutException, ElementClickInterceptedException) as e:
        print(f"No se encontró el botón 'Mostrar más' después de esperar: {e}")
        return False
    except Exception as e:
        print(f"Ocurrió un error al hacer clic en el botón 'Mostrar más': {e}")
        return False

def obtener_datos_articulos(busqueda):
    """
    Realiza una búsqueda en el sitio web de Mas Online y obtiene los datos de los artículos.

    :param busqueda: Término de búsqueda.
    :return: DataFrame con los datos de los artículos.
    """
    # Configura Selenium con undetected_chromedriver
    options = uc.ChromeOptions()
    browser = uc.Chrome(options=options, driver_executable_path=ChromeDriverManager().install())

    
    try:
        # Realiza la búsqueda
        url_busqueda = f'https://www.masonline.com.ar/268?initialMap=productClusterIds&initialQuery=268&map=category-2,productclusternames&query=/{busqueda}/mas-online---almacen&searchState'
        browser.get(url_busqueda)
        time.sleep(random.randint(10, 12))
        time.sleep(30)
        # Desplázate hacia abajo y haz clic en el botón "Mostrar más" si está presente
        xpath_del_boton = '//div[text()="Mostrar más" and contains(@class, "vtex-button__label") and contains(@class, "flex") and contains(@class, "items-center") and contains(@class, "justify-center") and contains(@class, "h-100") and contains(@class, "ph5")]'
        
        while True:
            # Espera para asegurar que la página está completamente cargada
            

            if not esperar_y_click_en_boton(browser, xpath_del_boton):
                break
            #scroll_to_bottom(browser, pause_time=10, step=1500)
            time.sleep(30)

            # Espera para que los nuevos elementos se carguen después de hacer clic
            time.sleep(random.randint(5, 8))
        scroll_to_bottom(browser, pause_time=3, step=1000)
        
        # Obtén el contenido HTML completo
        html = browser.page_source
        soup = BeautifulSoup(html, 'lxml')

        # Encuentra el número total de productos
        try:
            n_productos = int(soup.find('div', {'class': 'vtex-search-result-3-x-totalProducts--layout'}).text.strip().split(' ')[0])
        except AttributeError:
            n_productos = 0
        
        # Extrae los datos de los artículos
        galeria = soup.find('div', {'class': 'vtex-search-result-3-x-gallery'})
        articulo_name = galeria.find_all('span', {'class': 'vtex-product-summary-2-x-productBrand vtex-product-summary-2-x-brandName t-body'})
        articulo_nombres = [name.text.strip() for name in articulo_name]
        
        articulos_precios = galeria.find_all('div', {'class': 'valtech-gdn-dynamic-product-0-x-dynamicProductPrice mb4'})
        articulo_precios = [float(precio.text.replace('$', '').replace('.', '').replace(',', '.')) for precio in articulos_precios]
        
        elemento_precios =  galeria.find_all('div', {'class': 'vtex-product-price-1-x-priceSuspenseContentWrapper'})
        precios_antes = []
        
        for i in elemento_precios:
            valor = i.find('span', {'class': 'mt4 valtech-gdn-dynamic-product-0-x-weighableListPrice'})
            if valor == None:
                precios_antes.append(0.0)
            else:
                precios_antes.append(float(valor.text.replace('$', '').replace('.', '').replace(',', '.')))
                
                
        elemento_oferta =  galeria.find_all('div', {'class': 'vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--shelf-highlight'})
        oferta_antes = []

        for i in elemento_oferta:
            valor = i.find('span', {'class': 'highlight-undefined valtech-gdn-custom-highlights-0-x-customHighlightText'})
            if valor == None:
                oferta_antes.append('sin oferta')
            else:
                oferta_antes.append(valor.text)
        
        
        
                
                
                # Crea un DataFrame con los datos
        data = pd.DataFrame({'Articulo': articulo_nombres, 'Precio': articulo_precios, 'Precio Antes': precios_antes,'Ofertas':oferta_antes})
        
        # Verifica si el número de productos coincide
        if n_productos == data.shape[0]:
            return data
        else:
            print("Número de productos no coincide.")
            #return data
        
    finally:
        print(f'numeros de productos: {n_productos}, data: {data.shape[0]}')
        # Cierra el navegador
        browser.quit()
#datos_articulos = obtener_datos_articulos('Golosinas')

Este código automatiza el proceso de scraping y almacenamiento de datos de artículos en una página web.

Funcionamiento
- Guardar Datos:

Para cada categoría, se obtienen los datos de artículos y se guardan en archivos Excel. Los nombres de los archivos corresponden a las categorías.

- Completar Categorías Faltantes:

Si alguna categoría no se procesó inicialmente, se vuelve a intentar obtener y guardar sus datos.

-Procesar Todas las Categorías:

Procesa todas las categorías, manejando las que fallaron inicialmente. Se reintenta hasta que todas las categorías estén correctamente procesadas y guardadas en archivos.

**Este enfoque asegura que todos los datos sean correctamente extraídos y almacenados, manejando automáticamente cualquier error que pueda ocurrir durante el proceso.**

In [7]:
import os


def guardar_datos(categorias_lista):
    archivos_guardados = set()
    
    for categoria in categorias_lista:
        try:
            print(f"Procesando la categoría: {categoria}")
            
            # Obtén los datos de la categoría
            datos_articulos = obtener_datos_articulos(categoria)
            
            # Guarda los datos en un archivo Excel
            archivo_nombre = categoria + '.xlsx'
            datos_articulos.to_excel(archivo_nombre, index=False)
            
            archivos_guardados.add(archivo_nombre)
            
            print(f"Datos guardados en {archivo_nombre}")
            
        except Exception as e:
            print(f"Error al procesar la categoría '{categoria}': {e}")

    return archivos_guardados

def completar_categorias_faltantes(categorias_lista, archivos_guardados):
    archivos_guardados_nombres = {archivo.replace('.xlsx', '') for archivo in archivos_guardados}
    categorias_faltantes = {categoria for categoria in categorias_lista if categoria not in archivos_guardados_nombres}

    if categorias_faltantes:
        print("Categorías faltantes:", categorias_faltantes)
        print("Reintentando las categorías faltantes...")
        
        for categoria in categorias_faltantes:
            try:
                print(f"Procesando la categoría faltante: {categoria}")
                
                # Obtén los datos de la categoría
                datos_articulos = obtener_datos_articulos(categoria)
                
                # Guarda los datos en un archivo Excel
                archivo_nombre = categoria + '.xlsx'
                datos_articulos.to_excel(archivo_nombre, index=False)
                
                print(f"Datos guardados en {archivo_nombre}")
                
            except Exception as e:
                print(f"Error al procesar la categoría '{categoria}': {e}")

def procesar_todas_las_categorias(categorias_lista):
    categorias_no_procesadas = categorias_lista[:]
    while categorias_no_procesadas:
        print(f"Categorías restantes a procesar: {categorias_no_procesadas}")
        
        # Guarda datos de las categorías actuales
        archivos_guardados = guardar_datos(categorias_no_procesadas)
        
        # Completa las categorías faltantes
        completar_categorias_faltantes(categorias_no_procesadas, archivos_guardados)
        
        # Actualiza la lista de categorías no procesadas
        archivos_guardados_nombres = {archivo.replace('.xlsx', '') for archivo in archivos_guardados}
        categorias_no_procesadas = [categoria for categoria in categorias_no_procesadas if categoria not in archivos_guardados_nombres]

        if not categorias_no_procesadas:
            print("Todos los archivos han sido procesados.")
        else:
            print("Faltan categorías, reintentando...")


# Procesar todas las categorías
procesar_todas_las_categorias(categorias_lista)


Categorías restantes a procesar: ['Golosinas', 'Chocolates', 'Galletitas Dulces', 'Te', 'Yerbas', 'Alfajores', 'Cereales', 'Aderezos', 'Café', 'Mermeladas y untables', 'Especias', 'Sal Y Pimienta', 'Conservas', 'Pastas Secas', 'Endulzantes', 'Aceitunas Y Encurtidos', 'Tostadas, Grisines Y Marineras', 'Galletas De Arroz', 'Aceites', 'Chocolates, Rellenos Y Baños', 'Galletitas Saladas', 'Tomates Y Salsas', 'Harina De Trigo', 'Vinagres Y Aceto', 'Legumbres', 'Sabores En Polvo Y Bolsas Para Horno', 'Bizcochuelos, Brownies Y Tortas', 'Pan Rallado Y Rebozadores', 'Arroz', 'Tortas', 'Postres Y Flanes', 'Sopas', 'Salado', 'Leches Y Chocolatadas', 'Bizcochos', 'Premezclas', 'Pan Lactal', 'Donuts Y Facturas', 'Budines, Magdalenas Y Otros', 'Mas Reposteria', 'Snacks Salados', 'Gelatinas', 'Cacao', 'Pan Para Hamburguesas Y Panchos', 'Harina De Maiz', 'Caldos', 'Pan Árabe, Tortillas Y Otros', 'Pure', 'Panes Del Mundo', 'Budines, Magdalenas Y Piononos', 'Prepizzas Y Pizzetas', 'Otras Harinas', 'Oble

Este proyecto de data science ha demostrado la efectividad de aplicar técnicas de scraping para extraer información valiosa de una página web de supermercado. A través del proceso, hemos logrado obtener datos estructurados sobre productos, que incluyen nombres, precios y ofertas, organizados en archivos separados por categorías.

Las principales conclusiones son:

- Eficiencia en la Extracción de Datos: El uso de técnicas de scraping y herramientas automatizadas ha permitido una extracción eficiente de datos de una página web dinámica, superando las barreras de contenido cargado de forma asincrónica.

- Precisión en la Recolección de Información: La implementación de métodos para manejar el contenido dinámico y los botones de "Mostrar más" ha asegurado que todos los datos relevantes sean recolectados y almacenados correctamente.

- Valor de los Datos: Los datos obtenidos proporcionan una base sólida para análisis posteriores, como la evaluación de precios, la identificación de ofertas y la comparación de productos por categoría.

- Posibles Mejoras: Aunque el proceso ha sido exitoso, se podrían explorar mejoras en la robustez del scraping, como la gestión de posibles cambios en la estructura del sitio web y la inclusión de técnicas adicionales de limpieza y validación de datos.

En resumen, el proyecto ha alcanzado sus objetivos y ofrece una solución efectiva para la recolección de datos en un entorno web dinámico, sentando las bases para futuros análisis y aplicaciones en el campo de la ciencia de datos.